### Import Packages

In [27]:
import numpy as np
import pandas as pd
from datetime import datetime

%store -r df

### Data Preparation

#### Sort by Date

In [28]:
df = df.sort_values('date')

df.head(3)

,station_name,date,air_temp,wet_bulb_temp,humidity,rain_intensity,interval_rain,total_rain,precipitation_type,wind_direction,...,battery_life,year,month,day,hour,minute,weekday,weekofyear,season,timing
1068,63rd Street Weather Station,2015-04-25 09:00:00,7.0,5.9,86.0,7.2,5.0,5.2,60.0,119.0,...,12.0,2015,4,25,9,0,Saturday,17,Spring,Morning
1285,63rd Street Weather Station,2015-04-30 05:00:00,6.1,4.3,76.0,0.0,0.0,2.5,0.0,11.0,...,11.9,2015,4,30,5,0,Thursday,18,Spring,Morning
0,Oak Street Weather Station,2015-05-22 15:00:00,NaN,7.0,55.0,0.0,0.0,1.4,0.0,63.0,...,12.0,2015,5,22,15,0,Friday,21,Spring,Afternoon


#### Remove Unnecessary Columns

In [29]:
try:
  df = df.drop(columns=['minute', 'wet_bulb_temp', 
                        'rain_intensity', 'interval_rain', 
                        'total_rain', 'precipitation_type', 
                        'wind_direction', 'wind_speed', 
                        'max_wind_speed', 'barometric_pressure', 
                        'solar_radiation', 'heading'])
except Exception:
  pass

df.head(3)

,station_name,date,air_temp,humidity,battery_life,year,month,day,hour,weekday,weekofyear,season,timing
1068,63rd Street Weather Station,2015-04-25 09:00:00,7.0,86.0,12.0,2015,4,25,9,Saturday,17,Spring,Morning
1285,63rd Street Weather Station,2015-04-30 05:00:00,6.1,76.0,11.9,2015,4,30,5,Thursday,18,Spring,Morning
0,Oak Street Weather Station,2015-05-22 15:00:00,NaN,55.0,12.0,2015,5,22,15,Friday,21,Spring,Afternoon


#### Add t-1 Column

Necessary to detect unavailable data.

In [30]:
dfs = [df[df['station_name']=='Oak Street Weather Station'], df[df['station_name']=='Foster Weather Station'], df[df['station_name']=='63rd Street Weather Station']]

df['status'] = 'online'
df['air_temp_t-1'] = np.nan
df['humidity_t-1'] = np.nan
df['battery_life_t-1'] = np.nan

for x in dfs:
  prev = None

  for index, row in x.iterrows():
    try:
      # if prev is set to None throw exception
      hour1 = prev['hour']
      hour2 = row['hour'] - 1

      if hour2 < 0:
        hour2 = 23
      
      if (hour1 == hour2):
        df.at[index, 'air_temp_t-1'] = prev['air_temp']
        df.at[index, 'humidity_t-1'] = prev['humidity']
        df.at[index, 'battery_life_t-1'] = prev['battery_life']
    except TypeError:
      pass

    prev = row

    if index > 10:
      break

df.head(10)

,station_name,date,air_temp,humidity,battery_life,year,month,day,hour,weekday,weekofyear,season,timing,status,air_temp_t-1,humidity_t-1,battery_life_t-1
1068,63rd Street Weather Station,2015-04-25 09:00:00,7.00,86.0,12.0,2015,4,25,9,Saturday,17,Spring,Morning,online,NaN,NaN,NaN
1285,63rd Street Weather Station,2015-04-30 05:00:00,6.10,76.0,11.9,2015,4,30,5,Thursday,18,Spring,Morning,online,NaN,NaN,NaN
0,Oak Street Weather Station,2015-05-22 15:00:00,NaN,55.0,12.0,2015,5,22,15,Friday,21,Spring,Afternoon,online,NaN,NaN,NaN
180,Foster Weather Station,2015-05-22 16:00:00,9.17,59.0,15.1,2015,5,22,16,Friday,21,Spring,Afternoon,online,NaN,NaN,NaN
225,Foster Weather Station,2015-05-22 17:00:00,9.28,61.0,15.1,2015,5,22,17,Friday,21,Spring,Evening,online,NaN,NaN,NaN
1,Oak Street Weather Station,2015-05-22 17:00:00,NaN,56.0,12.1,2015,5,22,17,Friday,21,Spring,Evening,online,NaN,NaN,NaN
2,Oak Street Weather Station,2015-05-22 18:00:00,NaN,54.0,12.1,2015,5,22,18,Friday,21,Spring,Evening,online,NaN,56.0,12.1
205,Foster Weather Station,2015-05-22 18:00:00,9.89,52.0,15.1,2015,5,22,18,Friday,21,Spring,Evening,online,NaN,NaN,NaN
3,Oak Street Weather Station,2015-05-22 19:00:00,NaN,53.0,12.1,2015,5,22,19,Friday,21,Spring,Evening,online,NaN,54.0,12.1
213,Foster Weather Station,2015-05-22 19:00:00,9.56,58.0,15.1,2015,5,22,19,Friday,21,Spring,Evening,online,NaN,NaN,NaN


#### Add Datetime

In [32]:
dfs = [df[df['station_name']=='Oak Street Weather Station'], df[df['station_name']=='Foster Weather Station'], df[df['station_name']=='63rd Street Weather Station']]

if not 'status' in df: df['status'] = 'online'
if not 'air_temp_t-1' in df: df['air_temp_t-1'] = np.nan
if not 'humidity_t-1' in df: df['humidity_t-1'] = np.nan
if not 'battery_life_t-1' in df: df['battery_life_t-1'] = np.nan

for x in dfs:
    
    prev = None

    for index, row in x.iterrows():
        try:
            # if prev is set to None throw exception
            d1 = prev['date']
            d2 = row['date']

            diff = int((d2 - d1).seconds / 3600)

            if (diff > 1):
                for i in range(1, diff):
                    if i == 1:
                        data_prev = {
                            'air_temp_t-1': prev['air_temp'],
                            'humidity_t-1': prev['humidity'],
                            'battery_life_t-1': prev['battery_life']
                        }
                    else:
                        data_prev = {}

                    data = {
                        'station_name': row['station_name'],
                        'date': d1 + pd.DateOffset(hours=i),
                        'status': 'offline'
                    }

                    data.update(data_prev)

                    df_insert = pd.DataFrame.from_records([data])
                    df = df.append(df_insert, ignore_index=True)

        except TypeError:
            pass

        prev = row

        if index > 10:
            break

df = df.sort_values('date')
df.head(30)

,station_name,date,air_temp,humidity,battery_life,year,month,day,hour,weekday,weekofyear,season,timing,status,air_temp_t-1,humidity_t-1,battery_life_t-1
0,63rd Street Weather Station,2015-04-25 09:00:00,7.00,86.0,12.0,2015.0,4.0,25.0,9.0,Saturday,17.0,Spring,Morning,online,NaN,NaN,NaN
62280,63rd Street Weather Station,2015-04-25 10:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,offline,7.0,86.0,12.0
62281,63rd Street Weather Station,2015-04-25 11:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,offline,NaN,NaN,NaN
62282,63rd Street Weather Station,2015-04-25 12:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,offline,NaN,NaN,NaN
62283,63rd Street Weather Station,2015-04-25 13:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,offline,NaN,NaN,NaN
62284,63rd Street Weather Station,2015-04-25 14:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,offline,NaN,NaN,NaN
62285,63rd Street Weather Station,2015-04-25 15:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,offline,NaN,NaN,NaN
62286,63rd Street Weather Station,2015-04-25 16:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,offline,NaN,NaN,NaN
62287,63rd Street Weather Station,2015-04-25 17:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,offline,NaN,NaN,NaN
62288,63rd Street Weather Station,2015-04-25 18:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,offline,NaN,NaN,NaN
